# Hotel-Reservations-Prediction

## Import

In [1]:
import pandas as pd
import os

from typing import Text
from absl import logging

from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

from modules.components import init_components

## Ubah label data & drop kolom yang tidak dipakai

In [19]:
# df = pd.read_csv('data/Hotel Reservations.csv', encoding='latin-1')

# kolom_gak_pake = ['Booking_ID', 'arrival_year', 'arrival_month', 'arrival_date']

# df.drop(columns=[col for col in kolom_gak_pake if col in df.columns], inplace=True)

# def ganti_nama_fitur(fitur):
#     return 0 if fitur=='Canceled' else 1

# if df['booking_status'].dtype == object:
#     df['booking_status'] = [ganti_nama_fitur(x) for x in df['booking_status']]

# #df.drop(columns=['Unnamed: 0'], inplace=True)
# df.to_csv('data/Hotel Reservations_utf8.csv', index=False, encoding='utf-8')
# df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,Offline,0,0,0,65.00,0,1
1,2,0,2,3,Not Selected,0,Room_Type 1,5,Online,0,0,0,106.68,1,1
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,Online,0,0,0,60.00,0,0
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,Online,0,0,0,100.00,0,0
4,2,0,1,1,Not Selected,0,Room_Type 1,48,Online,0,0,0,94.50,0,0


## Set Variabel

In [24]:
PIPELINE_NAME = "axelseancp-pipeline"

DATA_ROOT = os.path.join(os.getcwd(), 'data')
TRANSFORM_MODULE_FILE = "modules/hotel_transform.py"
TRAINER_MODULE_FILE = "modules/hotel_trainer.py"
TUNER_MODULE_FILE = "modules/hotel_tuner.py"

OUTPUT_BASE = "outputs"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

## Run Pipeline

In [42]:
from tfx.orchestration import metadata, pipeline

def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing",
        # 0 auto-detect based on on the number of CPUs available 
        # during execution time.
        "--direct_num_workers=0" 
    ]
    
    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        # beam_pipeline_args=beam_args # Error: The order of analyzers in your preprocessing_fn appears to be non-deterministic. 
        # This can be fixed either by changing your preprocessing_fn such that tf.Transform analyzers are encountered 
        # in a deterministic order or by passing a unique name to each analyzer API call
    )

In [43]:
components = init_components(
    DATA_ROOT,
    tuner_module=TUNER_MODULE_FILE,
    training_module=TRAINER_MODULE_FILE,
    transform_module=TRANSFORM_MODULE_FILE,
    training_steps=5000,
    eval_steps=1000,
    serving_model_dir=serving_model_dir,
)

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

Trial 10 Complete [00h 00m 06s]
val_binary_accuracy: 0.7799000144004822

Best val_binary_accuracy So Far: 0.7964000105857849
Total elapsed time: 00h 01m 32s
Results summary
Results in outputs\axelseancp-pipeline\Tuner\.system\executor_execution\90\.temp\90\hotel-reservations-prediction
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 03 summary
Hyperparameters:
units: 64
learning_rate: 0.01
num_layers: 1
Score: 0.7964000105857849

Trial 07 summary
Hyperparameters:
units: 256
learning_rate: 0.01
num_layers: 2
Score: 0.7903000116348267

Trial 00 summary
Hyperparameters:
units: 64
learning_rate: 0.001
num_layers: 2
Score: 0.7893999814987183

Trial 05 summary
Hyperparameters:
units: 16
learning_rate: 0.001
num_layers: 3
Score: 0.7890999913215637

Trial 01 summary
Hyperparameters:
units: 64
learning_rate: 0.001
num_layers: 1
Score: 0.7878999710083008

Trial 08 summary
Hyperparameters:
units: 16
learning_rate: 0.01
num_layers: 1
Score: 0.7878000140190125



Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
